In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model 
%matplotlib inline

Дана статистика пользователей adult.csv.

Получите значения AUC для различных моделей и их параметров.

Решением будет ссылка на гитхаб с ноутбуком.


In [79]:
data=pd.read_csv('adult.csv')

In [80]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [81]:
data['income'][data['income']=='<=50K']=-1
data['income'][data['income']=='>50K']=1

/home/benito/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/benito/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [82]:
X=data.iloc[:,:-1]
y=data['income']
y=y.astype('int')

In [83]:
cat_feat = list(X.dtypes[data.dtypes == object].index)
X[cat_feat] = X[cat_feat].fillna('nan')
cat_feat_full = list(X.dtypes[X.dtypes == object].index)
cat_nunique = X[cat_feat_full].nunique()
cat_feat = list(cat_nunique[cat_nunique < 10].index)
dummy_X = pd.get_dummies(X[cat_feat], columns=cat_feat)
dummy_X.head()

,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital-status_Divorced,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male
0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0


In [84]:
num_feat = [f for f in X if f not in (cat_feat_full + ['Id'])]
scaler = StandardScaler()
scaler.fit(X[num_feat])
X_median = X[num_feat].median()
X[num_feat] = X[num_feat].fillna(X_median)
X[num_feat].head()

/home/benito/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
4,18,103497,10,0,0,30


In [85]:
X=pd.concat([X[num_feat], dummy_X], axis=1)
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male
0,25,226802,7,0,0,40,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,18,103497,10,0,0,30,1,0,0,0,...,1,0,0,0,0,0,0,1,1,0


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Дерево решений

In [87]:
rfc= RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)

In [88]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [89]:
y_pred=rfc.predict(X_test)

In [92]:
roc_auc_score( y_test, y_pred)

0.7449876814694334

In [116]:
rfc_1= RandomForestClassifier(n_estimators=10, max_depth=2, min_samples_leaf=20, max_features=0.5, n_jobs=-1)

In [118]:
rfc_1.fit(X_train,y_train)
y_pred=rfc_1.predict(X_test)
roc_auc_score( y_test, y_pred)

0.7140088233202552

In [119]:
rfc_2= RandomForestClassifier(n_estimators=20, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)

In [120]:
rfc_2.fit(X_train,y_train)
y_pred=rfc_2.predict(X_test)
roc_auc_score( y_test, y_pred)

0.746760039102963

### Логистическая регрессия

In [95]:
logr = LogisticRegression()

In [96]:
logr.fit(X_train,y_train)
y_pred=logr.predict(X_test)
roc_auc_score( y_test, y_pred)

/home/benito/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6152321003485376

In [100]:
logr_l1 = LogisticRegression(penalty='l1', C=0.1)

In [101]:
logr_l1.fit(X_train,y_train)
y_pred=logr.predict(X_test)
roc_auc_score( y_test, y_pred)

/home/benito/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6152321003485376

In [103]:
logr_l2 = LogisticRegression(penalty='l2', C=0.1)

In [104]:
logr_l2.fit(X_train,y_train)
y_pred=logr.predict(X_test)
roc_auc_score( y_test, y_pred)

/home/benito/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6152321003485376

### Линейная регрессия

In [109]:
lin=sklearn.linear_model.SGDClassifier(loss='log', random_state=1)

In [110]:
lin.fit(X_train,y_train)
y_pred=lin.predict(X_test)
roc_auc_score( y_test, y_pred)

/home/benito/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.5

In [111]:
lin_l=sklearn.linear_model.Lasso()

In [112]:
lin_l.fit(X_train,y_train)
y_pred=lin_l.predict(X_test)
roc_auc_score( y_test, y_pred)

0.7778822235441947

In [113]:
lin_r=sklearn.linear_model.Ridge()

In [114]:
lin_r.fit(X_train,y_train)
y_pred=lin_r.predict(X_test)
roc_auc_score( y_test, y_pred)

0.8903973092283967